# Trajectory Intention Classifiers and Novelty Detector script

This script provides step-step procedure to predict the trajectory intention classes of drones. First, the data pre-processing to transform the data into sub-trajectory features in accordance with a pre-selected window length is provided. Then, the data is splited into training, validation and testing datasets to train and test our models. Here, we test several neural models for classification and trajectory reconstruction. 
The models used for classification are:
1. Random Forest (Baseline)
2. LSTM: Long Short Term Memory
3. GRU: Gated Recurrent Unit
4. CBLSTM: Convolutional Biditectional-LSTM
5. CBLSTMA: CBLSTM with Attention
6. CNN: Convolutional Neural Network
7. CNNA: CNN with Attention

The models used for reconstruction are:
1. LSTM Autoencoder
2. Convolutional Autoencoder
3. CLSTM Autoencoder

The final hybrid model joins both models with
1. Classifier - CBLSTMA
2. Reconstruction- LSTM Autoencoder

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import seaborn as sns
import tensorflow as tf

# Import Packages from our Framework
from Framework.DataGeneration.DataPreprocessing import Preprocessing
from Framework.DataGeneration.Standardiser import TrajectoryStandardiser
from Framework.HybridClassifier.Visualization import Results
from Framework.HybridClassifier.RandomForest_Classifier import RandomForest
from Framework.HybridClassifier.LSTM_Classifier import LSTM_Network
from Framework.HybridClassifier.GRU_Classifier import GRU_Network
from Framework.HybridClassifier.ConvLSTM_Classifier import CLSTM_Network
from Framework.HybridClassifier.Convolutional_Classifier import Conv_Network
from Framework.HybridClassifier.Transformer_Classifier import Transformer_Network
from Framework.HybridClassifier.CLSTM_Attention_Classifier import CLSTMA_Network
from Framework.HybridClassifier.LSTM_Autoencoder import LSTM_Autoencoder
from Framework.HybridClassifier.Convolutional_Autoencoder import Convolutional_Autoencoder
from Framework.HybridClassifier.Convolutional_Classifier_Autoencoder import Conv_AE_Classifier
from Framework.HybridClassifier.CLSTM_Autoencoder import CLSTM_AE

from datetime import datetime
from time import time
from tqdm import tqdm

In [ ]:
# set random seeds - ensure consistent results
from numpy.random import seed
seed(12)
from tensorflow.random import set_seed
set_seed(12)

In [ ]:
# For pretty visualisations of the plots
sns.set_style('white')
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams['text.usetex'] = True
plt.rc('xtick', labelsize = 16) 
plt.rc('ytick', labelsize = 16)
plt.rcParams['font.size'] = 16 
plt.rcParams['lines.linewidth'] = 2
plt.rc('savefig', dpi=300)
plt.rc('axes', titlesize = 16, labelsize = 16)
%matplotlib inline

In [ ]:
# get project path and set data directory for notebook
PROJECT_PATH = os.getcwd()

DATA_DIR = os.path.join(PROJECT_PATH, 'ResearchData/simulated_measurements')
DEST_FOLDER = os.path.join(PROJECT_PATH,'FinalModels/Classification')
isExist = os.path.exists(DEST_FOLDER)
if not isExist:
    # Create a new directory because it does not exist
    os.makedirs(DEST_FOLDER)
    print("The new directory is created!")

In [ ]:
# define intent classes used in this modelling
CLASSES = ['mapping_flight', 'package_delivery', 
           'perimeter_flight', 'point_point_flight']

In [ ]:
# final dataframe to store all modelling results
final_results_df = pd.DataFrame()

# also store final hybrid model results (including recons)
final_hybrid_results_df = pd.DataFrame()

# Data Pre-processing

## Preprocessing strategy

$\bullet$ Load every flight trajectory track, split into sub-trajectories of equal window length, and build input feature tensor with associated labels.

$\bullet$ Concatenate all flights together in this way, into a unified dataset of input features with associated labels.

$\bullet$ Create a training and test split, so that the original trajectories (from which the sub-trajectories are taken) are unique between each split. Basically, we don't want to be predicting flights already seen (or partially seen) within the training data, for better estimations of generalisation performance.

$\bullet$ Center Cartesian coordinates of all sub-trajectories so that the first instance starts at $(0, 0, 0)$.

$\bullet$ Standardise numerical features so that they are z-scaled using the mean and standard deviation of the entire training set.

$\bullet$ Label encode uav_type feature to 0 for 'LSS' and 1 for 'Fixed-Wing'.

In [ ]:
# define tracking features to use within our modelling for classification
KEEP_COLS = ['est_x', 'est_y', 'est_z', 'est_vel_x', 'est_vel_y', 'est_vel_z',
             'uav_type']

Preprocessor = Preprocessing()
Plots = Results()

# set uav_type idx below (incl zero index)
UAV_TYPE_IDX = 6

# Window size 8, 16, 32, 64
WINDOW_SIZE = 8 # random seed data split=12

# new window every 50% into old window
if WINDOW_SIZE < 64:
    OVERLAP_FACTOR = 2 # use for windows 8, 16 & 32
else:
    OVERLAP_FACTOR = 4 # use for longer sequences (64)

# define encoding mappers for our categorical features
UAV_TYPE_MAP = {'LSS' : 0, 
                'Fixed-Wing' : 1}

UAV_INTENT_MAP = {'mapping_flight' : 0, 
                  'package_delivery' : 1,
                  'perimeter_flight' : 2, 
                  'point_point_flight' : 3}

ID_TO_INTENT_MAP = {v:k for k,v in UAV_INTENT_MAP.items()}

In [ ]:
# Generate sub-trajectories based on the proposed window size

X0 = []
y0 = []
flight_refs0 = []

# iterate each class and generate sub-trajectories for flights
for class_name in tqdm(CLASSES):
    cls_trajs, cls_labels, cls_refs = Preprocessor.get_class_trajectories(class_name, 
                                                                           DATA_DIR, 
                                                                           KEEP_COLS, 
                                                                           WINDOW_SIZE,
                                                                           OVERLAP_FACTOR)
    X0.append(cls_trajs)
    y0.append(cls_labels)
    flight_refs0.append(cls_refs)

# convert overall results into numpy arrays
X0 = np.concatenate(X0)
y0 = np.concatenate(y0)
flight_refs0 = np.concatenate(flight_refs0)
print(f"X shape: {X0.shape} \ny shape: {y0.shape}\nFlight refs: {flight_refs0.shape}")

In [ ]:
# assess number trajectories for each label and plot
cls_counts = np.unique(y0, return_counts=True)

plt.figure(figsize=(10,4))
plt.bar(x=cls_counts[0], height=cls_counts[1], color=['tab:blue', 'tab:green',
                                                      'tab:orange', 'tab:red'])
plt.title("Trajectory Intention Class Counts")
plt.ylabel("Count")
plt.show()

In [ ]:
# Remove outliers from the trajectories in accordance with a threshold
ANOMALOUS_THRESHOLD = 11000
X, y, flight_refs = Preprocessor.OutliersRemoval(UAV_TYPE_IDX, X0, y0, flight_refs0, ANOMALOUS_THRESHOLD)
print(f"X shape: {X.shape} \ny shape: {y.shape}\nFlight refs: {flight_refs.shape}")

In [ ]:
# Train, validation and test split to create the training, validation and test sets
random_seed = 12
test_size = 0.10
val_size = 0.20

(X_train, y_train, X_test, y_test, X_val, y_val, 
 train_flight_refs, val_flight_refs, test_flight_refs) = Preprocessor.Train_Val_Test_Split(X,
                                                                                           y,
                                                                                           flight_refs,
                                                                                           random_state = random_seed,
                                                                                           test_size = test_size,
                                                                                           val_size = val_size)

In [ ]:
# assess number trajectories for each label and plot
trg_cls_counts = np.unique(y_train, return_counts=True)
ticks = [x for x in range(len(trg_cls_counts[0]))]

fig, ax = plt.subplots(1,3, figsize=(14,4))
ax[0].bar(x=trg_cls_counts[0], height=trg_cls_counts[1], 
          color=['tab:blue', 'tab:green', 'tab:orange', 'tab:red'])
ax[0].set_title("Class counts (Training)", weight="bold")
ax[0].set_xticks(ticks=ticks, labels=trg_cls_counts[0], 
                 rotation=45, ha='right')

# assess number trajectories for each label and plot
val_cls_counts = np.unique(y_val, return_counts=True)
ax[1].bar(x=val_cls_counts[0], height=val_cls_counts[1], 
          color=['tab:blue', 'tab:green', 'tab:orange', 'tab:red'])
ax[1].set_title("Class counts (Validation)")
ax[1].set_xticks(ticks=ticks, labels=val_cls_counts[0], 
                 rotation=45, ha='right')

# assess number trajectories for each label and plot
test_cls_counts = np.unique(y_test, return_counts=True)
ax[2].bar(x=test_cls_counts[0], height=test_cls_counts[1], 
          color=['tab:blue', 'tab:green', 'tab:orange', 'tab:red'])
ax[2].set_title("Class counts (Test)")
ax[2].set_xticks(ticks=ticks, labels=test_cls_counts[0], 
                 rotation=45, ha='right')
plt.show()

## Data Standardisation


We also need to standardise our features so that they are scaled appropriately for our modelling later on. To do this, we'll use the means and standard deviations for each feature across all timesteps to standardise each feature to have approximately zero mean and 1 standard deviation. This will only be approximate however, given the time-series nature of the trajectories, which makes standardising effectively more complicated.

Note that our standardiser object below will return two sets of features:

$\bullet$ Trajectory features: these are multivariate time-series features of target track features for each sub-trajectory.

$\bullet$ Summary (meta) features: these summarise the sub-trajectory using means, standard deviations, min and max point-based (row for each label) features.

Within this classification work, we only use the first set of features (trajectory time-series). 

In [ ]:
# instantiate our scaler and fit_tranform training, transform test
standardiser = TrajectoryStandardiser(num_upper_idx=UAV_TYPE_IDX, 
                        cat_idx=[UAV_TYPE_IDX],
                        cat_mappers=[UAV_TYPE_MAP])

In [ ]:
# get standardised sequences and categorical feats
X_train_std, X_train_meta = standardiser.fit_transform(X_train)
X_val_std, X_val_meta = standardiser.transform(X_val)
X_test_std, X_test_meta = standardiser.transform(X_test)
print(f"Training: \n - Seqs: {X_train_std.shape}\n - Meta: {X_train_meta.shape}")
print(f"\nValidation: \n - Seqs: {X_val_std.shape}\n - Meta: {X_val_meta.shape}")
print(f"\nTest: \n - Seqs: {X_test_std.shape}\n - Meta: {X_test_meta.shape}")

In [ ]:
# Check if the training data possess aproximately mean zero
X_train_std[:, :UAV_TYPE_IDX, :].astype('float').mean(axis=(2,0))

In [ ]:
# Check if the training data possess standard deviation one
X_train_std[:, :UAV_TYPE_IDX, :].astype('float').std(axis=(2,0))

In [ ]:
# Gives the type of UAV that we are working with
uav_id_to_type = {v:k for k,v in UAV_TYPE_MAP.items()}
uav_id_to_type[np.argmax(X_train_meta[:, :2])]

In [ ]:
# Plot the real-trajectory and the processed trajectory
ex_idx = 6000
ex_uav_type = uav_id_to_type[np.argmax(X_train_meta[:, :2])]
fig, ax = plt.subplots(1,2, figsize=(12,4))
ax[0].scatter(X_train[ex_idx, 0, :], X_train[ex_idx, 1, :])
ax[0].plot(X_train[ex_idx, 0, :], X_train[ex_idx, 1, :], 
           alpha=0.3)
ax[0].scatter(X_train[ex_idx, 0, 0], X_train[ex_idx, 1, 0],
              label='Start', marker='*', color='tab:green', s=200)
ax[0].set_title(f"Sub-Traj ({y_train[ex_idx]})\nUAV: {ex_uav_type}")
ax[0].set_xlabel('x', weight='bold')
ax[0].set_ylabel('y', weight='bold')
ax[0].grid(0.5)

ax[1].scatter(X_train_std[ex_idx, 0, :], X_train_std[ex_idx, 1, :])
ax[1].plot(X_train_std[ex_idx, 0, :], X_train_std[ex_idx, 1, :], 
           alpha=0.3)
ax[1].scatter(X_train_std[ex_idx, 0, 0], X_train_std[ex_idx, 1, 0],
              label='Start', marker='*', color='tab:green', s=200)
ax[1].set_title(f"Re-Centred Sub-Traj ({y_train[ex_idx]})\nUAV: {ex_uav_type}")
ax[1].set_xlabel('x', weight='bold')
ax[1].set_ylabel('y', weight='bold')
ax[1].grid(0.5)
plt.legend()
plt.show()

In [ ]:
# Define the labels of the trajectory intention classes
CLASS_MAPPINGS = {'mapping_flight' : 0,
                  'package_delivery' : 1,
                  'perimeter_flight' : 2,
                  'point_point_flight' : 3}

In [ ]:
# get one hot encoded labels
y_train_oh = standardiser.preprocess_labels(y_train, CLASS_MAPPINGS)
y_val_oh = standardiser.preprocess_labels(y_val, CLASS_MAPPINGS)
y_test_oh = standardiser.preprocess_labels(y_test, CLASS_MAPPINGS)
y_train_oh.shape, y_val_oh.shape, y_test_oh.shape

In [ ]:
# we'll also get normal encoded labels too, since they will be useful
# for evaluation of our results later
y_train_enc = standardiser.preprocess_labels(y_train, CLASS_MAPPINGS, one_hot=False)
y_val_enc = standardiser.preprocess_labels(y_val, CLASS_MAPPINGS, one_hot=False)
y_test_enc = standardiser.preprocess_labels(y_test, CLASS_MAPPINGS, one_hot=False)
y_train_enc.shape, y_val_enc.shape, y_test_enc.shape

In [ ]:
print(f"Final Train features: {X_train_std.shape}\nTrain labels: {y_train_oh.shape}")
print(f"\nFinal Val features: {X_val_std.shape}\nVal labels: {y_val_oh.shape}")
print(f"\nFinal Test features: {X_test_std.shape}\nTest labels: {y_test_oh.shape}")

## Random Forest (Baseline)

Before using Deep Learning methods, we'll make a relatively simple baseline, from which the performance of more complex models can be assessed.

We test a Random Forest classifier to make predictions of the intent class based on generated features from the sub-trajectory inputs. Since this model is not sequential (it only supports single point inputs), we need to take the sub-trajectory sequences and process them into a simplified format appropriate for this model. We'll simply generate a series of features from the given input sequences, which typically might include mean values, max / max values, standard deviation and others point-based summary features. This is the method employed by many classical approaches to trajectory classification.

In [ ]:

X_train_sim_std = standardiser.obtain_trajectory_features(X_train_std)
X_val_sim_std = standardiser.obtain_trajectory_features(X_val_std)
X_test_sim_std = standardiser.obtain_trajectory_features(X_test_std)
print("Standardised summary data shapes: ")
print(f"Train: {X_train_sim_std.shape} \nVal: {X_val_sim_std.shape} \nTest: {X_test_sim_std.shape}")

X_train_sim = standardiser.obtain_trajectory_features(X_train[:, :-1, :])
X_val_sim = standardiser.obtain_trajectory_features(X_val[:, :-1, :])
X_test_sim = standardiser.obtain_trajectory_features(X_test[:, :-1, :])
print("\nOriginal summary data shapes: ")
print(f"Train: {X_train_sim.shape} \nVal: {X_val_sim.shape} \nTest: {X_test_sim.shape}")

In [ ]:
# Create a sunoke random forest classifier
RF = RandomForest()

# Train the model with the summary features of the sub-trajectories
rf_clf, model_results = RF.Random_Forest(X_train_sim_std, y_train_enc,
                                 X_val_sim_std, y_val_enc,
                                 X_test_sim_std, y_test_enc,
                                 WINDOW_SIZE, n_estimators = 20)

# Save the final model in a Data Frame for future export
final_results_df = final_results_df.append(model_results, ignore_index = True)

In [ ]:
# Determine the feature importance of each feature in our model
classic_col_names = []
#for agg in ['mean', 'std', 'max', 'min']:
for agg in ['mean']:
    for x in KEEP_COLS[:-1]:
        classic_col_names.append(f"{agg}_{x}")

In [ ]:
# Small function to create a Data Frame of feature importance 
def feature_importances(rf_model, col_names):
    return pd.DataFrame({'columns' : col_names, 
                         'importance' : rf_model.feature_importances_}
                       ).sort_values('importance', ascending=False)

# Bar plot visualisation
importances = feature_importances(rf_clf, classic_col_names)
plt.figure(figsize=(10,5))
sns.barplot(x="columns", y="importance", data=importances)
plt.ylabel("Feature Importances", weight='bold')
plt.xlabel("Features", weight='bold')
plt.title("Random Forest Feature Importances", weight='bold')
plt.xticks(rotation=90)
plt.show()
print(importances)

## LSTM Classifier

In [ ]:
# Instantiate the LSTM network using our LSTM_Network class
lstm_model = LSTM_Network(epochs = 75, batch_size = 256)
lstm_model.LSTM_model(X_train_std).summary()

# Train the LSTM classifier
lstm_history = lstm_model.train(X_train_std, y_train_oh,
                                          X_val_std, y_val_oh)

In [ ]:
# Plot the results of the LSTM network
Plots.AccLoss(lstm_history)

# Results under the validation and testing sets
model_results = lstm_model.prediction(X_val_std, y_val_enc,
                                      X_test_std, y_test_enc,
                                      WINDOW_SIZE)

# Store the final results for future export
final_results_df = final_results_df.append(model_results, ignore_index=True)

## GRU Classifier

In [ ]:
# Instantiate the GRU network using our GRU Network class
gru_model = GRU_Network(epochs = 75, batch_size = 256)
gru_model.GRU_model(X_train_std).summary()

# Train the GRU classifier
gru_history = gru_model.train(X_train_std, y_train_oh,
                                          X_val_std, y_val_oh)

In [ ]:
# Plot the results of the LSTM Network
Plots.AccLoss(gru_history)

# Results under the validation and testing sets
model_results = gru_model.prediction(X_val_std, y_val_enc,
                      X_test_std, y_test_enc,
                      WINDOW_SIZE)

# Store the final results for future export
final_results_df = final_results_df.append(model_results, ignore_index=True)

## Convolutional Bidirectional LSTM (CBLSTM) Classifier

In [ ]:
# Define the hyperparameters of the network
convlstm_params = {'activation' : 'relu', 
                   'dense_units' : 64,
                   'dense_dropout' : 0.3,
                   'rnn_dropout' : 0.3,
                   'conv_filters' : 20,
                   'kernel_size' : 12,
                   'optimizer' : 'adam',
                   'bidirectional' : True,
                   'lstm_units' : 32,
                   'n_classes' : 4,
                   'lr' : 5e-4,
                  'epochs' : 75,
                  'batch_size' : 128}

# Instantiate our CLSTM network class with the pre-defined hyperparameters
convlstm_model = CLSTM_Network(**convlstm_params)
convlstm_model.ConvLSTM_model(X_train_std)

# Train our CLSTM/CBLSTM network 
convlstm_history = convlstm_model.train(X_train_std, y_train_oh, 
                                        X_val_std, y_val_oh)

In [ ]:
# Plot the results of the CLSTM/CBLSTM network
Plots.AccLoss(convlstm_history)

# Results under the validation and testing sets
model_results = convlstm_model.prediction(X_val_std, y_val_enc,
                      X_test_std, y_test_enc,
                      WINDOW_SIZE)

# Store the final result for testing
final_results_df = final_results_df.append(model_results, ignore_index=True)

## CNN/CNNA Classifier

In [ ]:
# Define the parameters of the network

conv_params = {'activation' : 'swish', 
               'dense_units' : 64,
               'dense_dropout' : 0.4,
               'filters' : [32,64],
               'kernel_size' : 12,
               'optimizer' : 'adam',
               'n_classes' : 4,
               'lr' : 1e-3,
               'attention' : True,
               'epochs': 85,
               'batch_size': 128
              }

# Instantiate the CNN/CNNA classifier
conv_model = Conv_Network(**conv_params)
conv_model.Conv_model(X_train_std).summary()

# Train the CNN/CNNA classifier
conv_history = conv_model.train(X_train_std, y_train_oh, 
                                        X_val_std, y_val_oh)

In [ ]:
# Plot the results of the CNN/CNNA classifier
Plots.AccLoss(conv_history)

# Results under the validation and testing sets
model_results = conv_model.prediction(X_val_std, y_val_enc,
                      X_test_std, y_test_enc,
                      WINDOW_SIZE)

# Store the results for future export
final_results_df = final_results_df.append(model_results, ignore_index=True)

## Attention-based Transformer classifier (not reported) 

Warning - this model is computationally intensive and unless running on GPU, will take a very long time. It is not recommended to run this on default CPU, especially on the lower window sizes of 8 and 16.

In [ ]:
# Instantiate the transformer network class
trans_model = Transformer_Network(epochs = 80)
trans_model.transformer_dnn_model(X_train_std)

# Train the transformer network
trans_history = trans_model.train(X_train_std, y_train_oh, 
                                        X_val_std, y_val_oh)

In [ ]:
# Plot the results of the transformer network
Plots.AccLoss(trans_history)

# Results under validation and testing sets
model_results = trans_model.prediction(X_val_std, y_val_enc,
                      X_test_std, y_test_enc,
                      WINDOW_SIZE)

# Store the results for future export
final_results_df = final_results_df.append(model_results, ignore_index=True)

The computational requirements for training, along with the poor performance compared to the Conv LSTM and Conv DNN variants, make this model a poor choice.

It is possible that with further tweaking, refinements and increased quantities of training data this model could perform significantly better, but even so, the extremely slow inference and training time on CPU makes it a poor choice currently.

## CLSTM/CBLSTM with Attention Classifier

In [ ]:
# Define the hyperparameters of the network
convlstm_att_params = {'activation' : 'swish',
                       'dense_units' : 64,
                       'dense_dropout' : 0.4,
                       'rnn_dropout' : 0.4,
                       'conv_filters' : 32,
                       'kernel_size' : 12,
                       'lr' : 5e-3,
                       'bidirectional' : True,
                       'lstm_units' : 20,
                       'n_classes' : 4,
                       'lr' : 5e-4}

# Instantiate the CLSTMA/CBLSTMA network
clstma_model = CLSTMA_Network(epochs = 100)
clstma_model.ConvLSTM_Attention_model(X_train_std)

# Train the CLSTMA/CBLSTMA network
clstma_history = clstma_model.train(X_train_std, y_train_oh, 
                                        X_val_std, y_val_oh)

In [ ]:
# Plot the results of the CLSTMA/CBLSTMA network
Plots.AccLoss(clstma_history)

# Results under validation and testing sets
model_results = clstma_model.prediction(X_val_std, y_val_enc,
                      X_test_std, y_test_enc,
                      WINDOW_SIZE)

# Store the final results for future export
final_results_df = final_results_df.append(model_results, ignore_index=True)

# Novelty Detection (Autoencoder - Based Reconstruction)

Within this section we'll explore Deep Autoencoders for Novelty Detection. This will allow us to determine whether new data coming in is similar or very different (anomalous) to that known and modelled by the system.

In effect, if new data coming in is flagged as anomalous, then this might correspond to a flight with unknown intent (does not fit clearly into one of our pre-defined categories). This could be useful, since we could then follow-up with a generic heuristic or modelling strategy to determine the future occupied space of that UAV (e.g. a generic particle filter with predictions n steps into the future).

## Sequential LSTM Autoencoder

In [ ]:
# Instantiate the LSTM Autoencoder
lstm_ae = LSTM_Autoencoder(epochs = 200)
lstm_ae.lstm_autoencoder_model(X_train_std).summary()

# Train the LSTM Autoencoder
lstm_ae_history = lstm_ae.train(X_train_std, X_val_std)

In [ ]:
# Plot the results of the LSTM Autoencoder 
Plots.RMSE(lstm_ae_history)

# Obtain the reconstructions, mean squared errors, and standard deviations of the training
# validation and testing datasets
trg_recon, trg_recon_mses, trg_recon_stds = lstm_ae.prediction(X_train_std)
val_recon, val_recon_mses, val_recon_stds = lstm_ae.prediction(X_val_std)
test_recon, test_recon_mses, test_recon_stds = lstm_ae.prediction(X_test_std)

In [ ]:
# Check the results of the autoencoder with unseen trajectories that do not match with our intent classes

# Preprocess the unseen trajectories
X_unseen, y_unseen, refs_unseen = Preprocessor.get_class_trajectories('other', 
                                                                      DATA_DIR, 
                                                                      KEEP_COLS, 
                                                                      WINDOW_SIZE,
                                                                      OVERLAP_FACTOR)

print(f"X shape: {X_unseen.shape} \ny shape: {y_unseen.shape}\n")

# preprocess unseen data, get recons using our autoencoder
X_unseen_std, X_unseen_meta = standardiser.transform(X_unseen)


In [ ]:
# Results of the LSTM autoencoder under the unseen trajectories
unseen_recons, unseen_recon_mses, unseen_recon_stds  = lstm_ae.prediction(X_unseen_std)
for idx in range(11):
    Plots.plot_recon_results(idx, X_unseen_std, y_unseen, unseen_recons, 
                       trg_recon_mses, val_recon_mses, figsize=(12,4))

## Sequential CNN Autoencoder

In [ ]:
# Instantiate the convolutional autoencoder class
conv_ae = Convolutional_Autoencoder(epochs = 200)
conv_ae.conv_autoencoder_model(X_train_std).summary()

# Train the convolutional autoencoder
conv_ae_history = conv_ae.train(X_train_std, X_val_std)

In [ ]:
# Results of the convolutional autoencoder
Plots.MSE(conv_ae_history)

# Obtain the reconstructions, mean squared errors, and standard deviations of the training
# validation and testing datasets
trg_conv_ae_recon, trg_conv_ae_recon_mses, trg_conv_ae_recon_stds = conv_ae.prediction(X_train_std)
val_conv_ae_recon, val_conv_ae_recon_mses, val_conv_ae_recon_stds = conv_ae.prediction(X_val_std)
test_conv_ae_recon, test_conv_ae_recon_mses, test_conv_ae_recon_stds = conv_ae.prediction(X_test_std)

In [ ]:
# Results of the CNN autoencoder under the unseen trajectories
unseen_recons, unseen_error_mean, unseen_error_stds  = conv_ae.prediction(X_unseen_std)

print(f"Unseen (other) mean feature MSE: {unseen_error_mean}")
print(f"Times larger than training mean: {unseen_error_mean / trg_conv_ae_recon_mses}")

# Combined Autoencoder Novelty Detector and Classifier

We'll create a hybrid architecture that performs both novelty detection and trajectory classification. This will build on all of the techniques covered above, designed into a unified classification / novelty detector architecture.

Novelty detection will be performed using an Autoencoder component of the network, whilst trajectory classification will be performed using a custom dense output from the encoder part of the autoencoder.

The benefit of such an architecture is that it means we have a single model that performs both novelty detection and classification, rather than wastefully having two large DNNs.


## Convolutional Classifier & Autoencoder 

In [ ]:
# Instantiate the convolutional classifier & autoencoder novelty detector
conv_ae_class = Conv_AE_Classifier(epochs = 75)
conv_ae_class.Conv_AE_Classifier(X_train_std).summary()

# Train the classifier & autoencoder novelty detector
conv_ae_class_history = conv_ae_class.train(X_train_std, y_train_oh, X_val_std, y_val_oh)

In [ ]:
# Results of both the classifier and novelty detector
Plots.Custom_AccLoss(conv_ae_class_history)

# Obtain the reconstructions, mean squared errors, and standard deviations of the training
# validation, testing and unseen datasets
trg_preds, trg_recons, trg_recon_mse, trg_recon_std = conv_ae_class.prediction(X_train_std, y_train_enc)
val_preds, val_recons, val_recon_mse, val_recon_std = conv_ae_class.prediction(X_val_std, y_val_enc)
test_preds, test_recons, test_recon_mse, test_recon_std = conv_ae_class.prediction(X_test_std, y_test_enc)
unseen_preds, unseen_recons, unseen_recon_mse, unseen_recon_std = conv_ae_class.predictionUnseen(X_unseen_std)
print(f"Unseen MSE factor relative to training: {unseen_recon_mse/trg_recon_mse}")

## CLSTM/CBLSTMA & Autoencoder

In [ ]:
# Define the hyperparameters of the CLSTM/CBLSTMA & LSTM Autoencoder
clstm_ae_params = {'activation' : 'swish',
                      'dense_units' : 64,
                      'dense_dropout' : 0.4,
                      'rnn_dropout' : 0.4,
                      'conv_filters' : [32],
                      'kernel_size' : 12,
                      'lr' : 1e-3,
                      'bidirectional' : True,
                      'attention' : True,
                      'lstm_units' : [20, 20],
                      'n_classes' : 4,
                      'clf_model_weight' : 0.95,
                      'codings_size' : 16,
                      'epochs': 150,
                      'batch_size': 128}

# Instantiate the CLSTM/CBLSTMA & Autoencoder class
clstm_ae = CLSTM_AE(**clstm_ae_params)
clstm_ae.CLSTM_AE(X_train_std).summary()

# Train the CBLSTMA & LSTM Autoencoder network
clstm_ae_history = clstm_ae.train(X_train_std, y_train_oh, X_val_std, y_val_oh, y_train_enc, WINDOW_SIZE)

In [ ]:
# Results for validation and testing sets for classification and reconstruction
(model_results, val_preds, 
 val_recons, test_preds, test_recons,
 trg_recon_mse, val_recon_mse, test_recon_mse) = clstm_ae.prediction(X_val_std, y_val_enc,
                                                                                  X_test_std, y_test_enc,
                                                                                  WINDOW_SIZE)

# Store the results for future export
final_results_df = final_results_df.append(model_results, ignore_index=True)
ae_hybrid_results = model_results.copy()
ae_hybrid_results['Train Recon MSE'] = trg_recon_mse
ae_hybrid_results['Val Recon MSE'] = val_recon_mse
ae_hybrid_results['Test Recon MSE'] = test_recon_mse

In [ ]:
# Reconstructions under unseen trajectories
unseen_preds, unseen_recons, unseen_recon_mse = clstm_ae.predictionUnseen(X_unseen_std)
ae_hybrid_results['Other Recon MSE'] = unseen_recon_mse

# Store the reconstruction results for future export
final_hybrid_results_df = final_hybrid_results_df.append(ae_hybrid_results, ignore_index=True)

In [ ]:
# Plot some unseen sample trajectories to observe the performance of the hybrid classifier
random_idx = np.random.choice(X_unseen_std.shape[0], 10, replace=False)
index = 0
for idx in random_idx:
    index += 1
    Plots.plot_classification_results(idx, index, X_unseen, X_unseen_std, y_unseen,
                                      unseen_preds, unseen_recons,
                                      trg_recon_mse, val_recon_mse,
                                      ID_TO_INTENT_MAP, UAV_INTENT_MAP)

In [ ]:
# Plot sample trajectories from the testing dataset to observe the performance of the hybrid classifier
random_idx = np.random.choice(X_test_std.shape[0], 10, replace=False)
for idx in random_idx:
    index += 1
    Plots.plot_classification_results(idx, index, X_test, X_test_std, y_test,
                                      test_preds, test_recons,
                                      trg_recon_mse, val_recon_mse,
                                      ID_TO_INTENT_MAP, UAV_INTENT_MAP)

In [ ]:
# lets also save the final model results 
now = datetime.now()
current_date = now.strftime("%Y%m%d")
final_results_df.to_excel(f"{current_date}-final_clf_results.xlsx", index=False)
final_hybrid_results_df.to_excel(f"{current_date}-final_hybrid_results.xlsx", index=False)

In [ ]:
# save model as .h5 file
model_name = f"clstm_ae_window_{WINDOW_SIZE}.h5"
model_savepath = os.path.join(DEST_FOLDER, model_name)
clstm_ae.save(model_savepath)